In [ ]:
!pip install -q datasets
!pip install -q git+https://github.com/huggingface/transformers
!pip install -q bitsandbytes sentencepiece accelerate loralib
!pip install -q -U git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 82.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from PIL import Image
from transformers import IdeficsForVisionText2Text, AutoProcessor, Trainer, TrainingArguments, BitsAndBytesConfig

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
#checkpoint = "HuggingFaceM4/idefics-9b"
checkpoint = "HuggingFaceM4/idefics-9b-instruct"

In [ ]:
bitsandbytes_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.float16,
    llm_int8_skip_modules = ["lm_head", "embed_tokens"]
)

In [ ]:
processor = AutoProcessor.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/283 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/92.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/207 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/


In [ ]:
model = IdeficsForVisionText2Text.from_pretrained(checkpoint, quantization_config = bitsandbytes_config, device_map = 'auto')

config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/99.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/7.89G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
model

IdeficsForVisionText2Text(
  (model): IdeficsModel(
    (embed_tokens): IdeficsDecoupledEmbedding(
      num_embeddings=32000, num_additional_embeddings=3, embedding_dim=4096, partially_freeze=False
      (additional_embedding): Embedding(3, 4096)
    )
    (vision_model): IdeficsVisionTransformer(
      (embeddings): IdeficsVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1280, kernel_size=(14, 14), stride=(14, 14), bias=False)
        (position_embedding): Embedding(257, 1280)
      )
      (pre_layrnorm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (encoder): IdeficsVisionEncoder(
        (layers): ModuleList(
          (0-31): 32 x IdeficsVisionEncoderLayer(
            (self_attn): IdeficsVisionAttention(
              (k_proj): Linear4bit(in_features=1280, out_features=1280, bias=True)
              (v_proj): Linear4bit(in_features=1280, out_features=1280, bias=True)
              (q_proj): Linear4bit(in_features=1280, out_features=1280, bias=True)
        

In [ ]:
#Inference

def do_inference(model, processor, prompts, max_new_tokens=300):
  tokenizer = processor.tokenizer
  bad_words = ["<image>", "<fake_token_around_image>"]
  if len(bad_words)>0:
    bad_words_ids = tokenizer(bad_words, add_special_tokens=False).input_ids
    eos_token = "</s>"
    eos_token_id = tokenizer.convert_tokens_to_ids(eos_token)

    inputs = processor(prompts, return_tensors="pt").to(device)
    generated_ids = model.generate(
        **inputs,
        eos_token_id = [eos_token_id],
        bad_words_ids = bad_words_ids,
        max_new_tokens = max_new_tokens,
        early_stopping = True
    )
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens = True)[0]
  return generated_text

In [ ]:
#Preprocessing
import torchvision.transforms as transforms

def convert_to_rgb(image):
  if image.mode == 'RGB':
    return image

  image_rgba = image.convert('RGBA')
  background = Image.new('RGBA', image_rgba.size, (255,255,255))
  alpha_comp = Image.alpha_composite(background, image_rgba)
  alpha_comp = alpha_comp.convert("RGB")
  return alpha_comp

def ds_transforms(example_batch):
  image_size = processor.image_processor.image_size
  image_mean = processor.image_processor.image_mean
  image_std = processor.image_processor.image_std

  image_transform = transforms.Compose([
      convert_to_rgb,
      transforms.RandomResizedCrop((image_size, image_size), scale = (0.9, 1.0), interpolation = transforms.InterpolationMode.BICUBIC),
      transforms.ToTensor(),
      transforms.Normalize(mean = image_mean, std = image_std)
  ])

  prompts = []
  for i in range(len(example_batch['caption'])):
    caption = example_batch['caption'][i].split(".")[0]
    prompts.append(
        [
            example_batch['url'][i],
            f"Question: What's on the picture? Answer:{caption}</s>",
        ],
    )

  inputs = processor(prompts, transform=image_transform, return_tensors='pt').to(device)
  inputs['labels'] = inputs["input_ids"]
  return inputs

In [ ]:
#ds = load_dataset("TheFusion21/PokemonCards")

ds = load_dataset("cat-state/mscoco-1st-caption")

#Make modifications to the dataset
#For example, remove rows containing a specific URL
# ds_modified = ds['train'].filter(lambda example: example['image_url'] != 'https://images.pokemontcg.io/mcd14/5_hires.png' and
#                                  example['image_url'] != 'https://images.pokemontcg.io/mcd14/3_hires.png' and
#                                  example['image_url'] != 'https://images.pokemontcg.io/mcd17/7_hires.png' and
#                                   example['image_url'] !='https://images.pokemontcg.io/mcd17/4_hires.png' and
#                                  example['image_url'] != 'https://images.pokemontcg.io/mcd18/1_hires.png' and
#                                  example['image_url'] !='https://images.pokemontcg.io/mcd18/5_hires.png' and
#                                   example['image_url'] != 'https://images.pokemontcg.io/mcd17/9_hires.png' and
#                                 example['image_url'] !="https://images.pokemontcg.io/mcd15/11_hires.png" and
#                                  example['image_url'] !="https://images.pokemontcg.io/mcd18/11_hires.png"
#                                   )
ds = ds["train"].train_test_split(test_size=0.002)
train_ds = ds["train"]
eval_ds = ds["test"]
train_ds.set_transform(ds_transforms)
eval_ds.set_transform(ds_transforms)

Resolving data files:   0%|          | 0/37 [00:00<?, ?it/s]

In [ ]:
model_name = checkpoint.split("/")[1]
config = LoraConfig(
    r = 16,
    lora_alpha = 32,
    target_modules = ["q_proj","k_proj","v_proj"],
    lora_dropout = 0.05,
    bias = "none"
)

In [ ]:
model = get_peft_model(model, config)

model.print_trainable_parameters()

trainable params: 19,750,912 || all params: 8,949,438,736 || trainable%: 0.22069442098698333


In [ ]:
training_args = TrainingArguments(
    output_dir = f"{model_name}-mscoco",
    learning_rate = 2e-4,
    fp16 = True,
    per_device_train_batch_size = 2,
    per_device_eval_batch_size = 2,
    gradient_accumulation_steps = 6,
    dataloader_pin_memory = False,
    save_total_limit = 3,
    evaluation_strategy = "steps",
    eval_steps = 10,
    save_steps = 50,
    max_steps = 50,
    logging_steps = 10,
    remove_unused_columns = False,
    label_names = ["labels"],
    load_best_model_at_end = True,
    report_to = "none",
    optim = "paged_adamw_8bit",
    num_train_epochs= 2

)

In [ ]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_ds,
    eval_dataset = eval_ds
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
10,2.186900,1.242024
20,0.953700,0.724383
30,0.705900,0.694027
40,0.714900,0.677315
50,0.675400,0.673307


TrainOutput(global_step=50, training_loss=1.0473335647583009, metrics={'train_runtime': 1546.3231, 'train_samples_per_second': 0.388, 'train_steps_per_second': 0.032, 'total_flos': 958378282137792.0, 'train_loss': 1.0473335647583009, 'epoch': 0.01})

In [ ]:
url = "https://thumbs.dreamstime.com/b/playa-esmeralda-holguin-cuba-view-top-beach-beautiful-caribbean-sea-turquoise-playa-esmeralda-holguin-114428373.jpg"

In [ ]:
prompts = [
    url,
    "Question: What's on the picture? Answer:"
]

In [ ]:
new = do_inference(model, processor, prompts, max_new_tokens=300)
print(new)

Question: What's on the picture? Answer:A beach with a blue sky and blue water
